In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def bipartite(G, N, M) :
  def dfs(x) :
    if vis[x]: return False

    vis[x] = True
    for y in G[x]:
      if sel[y] == -1 :
        sel[y] = x
        return True
      
    for y in G[x] :
      if dfs(sel[y]):
        sel[y] = x
        return True
    return False

  sel = [-1] * M
  res = 0
  for i in range(N) :
    vis = [False] * N
    if dfs(i) : res += 1
  return res, sel

# X = 88, . = 46
def sol() :
  N = int(input())
  chess = [[*input().rstrip()] for _ in range(N)]
  vis = [[0] * N for _ in range(N)]

  n = 0
  for i, l in enumerate(chess) :
    should_increase = True #.을 만났을 때 정점 번호를 늘려야 하는 조건
    for j, v in enumerate(l) :
      if v == 46 :
        if should_increase : 
          n += 1
          should_increase = not should_increase
        vis[i][j] = n
      else :
        should_increase = True
  
  vis2 = [[0] * N for _ in range(N)]
  m = 0
  for i, l in enumerate(zip(*chess)) :
    should_increase = True
    for j, v in enumerate(l) :
      if v == 46 :
        if should_increase : 
          m += 1
          should_increase = not should_increase
        vis2[i][j] = m
      else :
        should_increase = True
  
  vis2 = [*zip(*vis2)]
  
  G = [set() for _ in range(n)]
  for l1, l2 in zip(vis, vis2) :
    for i, j in zip(l1, l2) :
      if i and j :
        G[i-1].add(j-1)
  
  res, sel = bipartite(G, n, m)
  print(res)

sol()

### 풀이 및 구현
- 폰을 체스판에 배치하면 해당 열과 행에 다른 룩이 배치될 수 없다.
  - 이분 그래프의 관점에서 생각하면, 어떤 칸에 룩을 배치한다는 것은 `열` 기준으로 연결된 정점과 `행` 기준으로 연결된 정점끼리 매칭된 것으로 해석할 수 있다.
    - 이분 그래프이기 때문에 한번 매칭되면 다른 정점과는 매칭될 수 없는 성질을 이용한다.
- 이 문제에서 폰(벽이라고 생각해도 좋다)이 같은 열과 행에 다른 룩이 배치됐을때 서로 공격하지 않게 해주는 역할을 한다.\
이때 행과 열 번호를 폰을 기준으로 나눠야 하기 때문에 다음과 같은 기준으로 각 행과 열에 번호를 메긴다.
    - `x`를 정점 번호, `flag`를 다음 `.`을 만났을 때 정점 번호를 바꿀지 여부를 결정하는 변수라고 했을 때, 각 cell을 순회하면서 다음을 수행한다.
      - 한 행이 끝나면 `flag`를 `True`로 바꾼다.
      - `X`를 만나면 `flag`를 `True`로 바꾼다.
      - `.`을 만났을 때 `flag`가 켜져있다면 `flag`를 `False`로 바꾸고 `x`를 1 증가시킨다.
    - 열에 대해서도 `zip(*G)`를 이용하여 그리드를 회전시킨 뒤, 위와 같은 방식으로 행과 열에 번호를 메긴다.
      - 또한 결과로 나온 넘버링 그리드는 또 다시 `zip(*vis)`해야 함에 주의.
- 넘버링한 두 그리드를 비교하면서 서로 넘버링된 cell에 대해 이분 그래프의 간선을 연결시킨다.
- 이분 매칭을 수행하고, 매칭된 정점의 개수를 출력한다.

5
X....
X....
..X..
.X...
....X
5
XXXXX
.....
XX...
.X...
....X

- set()쓰는게 좀 찝찝해서 bipartite를 수정하고, 인접 행렬을 처리하게 바꿨더니 TLE가 났다. 그정돈가?
  - dfs에서 각 정점을 확인하는데에 엄청난 시간이 걸리는게 원인일 것.